In [1]:
import sys
sys.path.append("..")

In [2]:
from dotenv import load_dotenv

In [3]:
_ = load_dotenv("../.env")
_ = load_dotenv("../.env.local")

In [4]:
from google import genai
from typing import List, Dict, Any
from pydantic import BaseModel, Field

In [5]:
client = genai.Client()

In [6]:
r = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=["hello", "what is today's date?"]
)

In [7]:
!ls | grep csv

assam.csv
chart_data.csv
odisha.csv


In [8]:
# file1 = client.files.upload(file="odisha.csv", 
#                            config={
#                                "name": "odisha-tcl",
#                                "display_name": "odisha tree cover loss"
#                            })

# file2 = client.files.upload(file="assam.csv", 
#                            config={
#                                "name": "assam-tcl",
#                                "display_name": "assam tree cover loss"
#                            })

In [8]:
for f in client.files.list():
    print(f.name, f.display_name)

files/assam-tcl assam tree cover loss
files/odisha-tcl odisha tree cover loss


In [9]:
file1 = client.files.get(name="files/odisha-tcl")
file2 = client.files.get(name="files/assam-tcl")

In [22]:
import pandas as pd
data1 = pd.read_csv("odisha.csv")
data2 = pd.read_csv("assam.csv")
data3 = pd.read_csv("karnataka.csv")

In [11]:
data1

,tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
0,Hard commodities,25966.917,1.761503e+07
1,Permanent agriculture,1820299.372,1.163551e+09
2,Unknown,3810.596,1.766994e+06
3,Wildfire,2187455.423,5.980315e+08
4,Logging,217133.258,2.542157e+08
5,Other natural disturbances,53478.238,2.216762e+07
6,Settlements & Infrastructure,5799.188,3.291979e+06
7,Shifting cultivation,81114.749,5.186880e+07


In [12]:
data2

,tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
0,Hard commodities,6.800,27.200
1,Permanent agriculture,99.372,72.738
2,Unknown,10.596,93.938
3,Wildfire,55.230,90.819
4,Logging,2.250,69.220
5,Other natural disturbances,478.238,21.170
6,Settlements & Infrastructure,9.180,979.210
7,Shifting cultivation,14.700,802.210


In [13]:
data1.to_csv(index=False)

'tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e\nHard commodities,25966.917,17615027.092\nPermanent agriculture,1820299.372,1163551272.738\nUnknown,3810.596,1766993.938\nWildfire,2187455.423,598031490.819\nLogging,217133.258,254215669.242\nOther natural disturbances,53478.238,22167621.917\nSettlements & Infrastructure,5799.188,3291979.121\nShifting cultivation,81114.749,51868802.621\n'

In [26]:
# prompt = f"""Compare deforestation by driver between Odisha & Assam.
# Odisha data as CSV:
# ```csv
# {data1.to_csv(index=False)}
# ```

# Assam data as CSV:
# ```csv
# {data2.to_csv(index=False)}
# ```
# """

# prompt = """Compare deforestation by driver between Odisha & Assam.

# ### Datasets
# - input_file_0.csv: Odisha, India - Tree cover loss data from 2020 - 2024
# - input_file_1.csv: Assam, India - Tree cover loss data from 2020 - 2024

# After analysis, write code to plot a bar chart of the analysis & save data for bar chart as a CSV - chart_data.csv file.
# """

prompt = """
### User Query:
Compare deforestation by driver between Odisha & Assam.

You have access to the following datasets (read-only):
- input_file_0.csv → Odisha, India — Tree cover loss (2020–2024)
- input_file_1.csv → Assam, India — Tree cover loss (2020–2024)
- input_file_2.csv -> Karnataka, India - Tree cover loss (2020-2024)
---

### Workflow:
1. **Analyze**: Use pandas to extract insights or summarize data relevant to the query. Print key findings clearly. Do **not** create any plots or charts.  
2. **Prepare output**: Recommend a suitable chart type for visualization (e.g., bar, line, pie). Create a clean DataFrame for the chart (x/y columns) and save it as: chart_data.csv.
3. **Summarize**: Summarize a data driven insights based on your analysis at the end.
"""

print(prompt)


### User Query:
Compare deforestation by driver between Odisha & Assam.

You have access to the following datasets (read-only):
- input_file_0.csv → Odisha, India — Tree cover loss (2020–2024)
- input_file_1.csv → Assam, India — Tree cover loss (2020–2024)
- input_file_2.csv -> Karnataka, India - Tree cover loss (2020-2024)
---

### Workflow:
1. **Analyze**: Use pandas to extract insights or summarize data relevant to the query. Print key findings clearly. Do **not** create any plots or charts.  
2. **Prepare output**: Recommend a suitable chart type for visualization (e.g., bar, line, pie). Create a clean DataFrame for the chart (x/y columns) and save it as: chart_data.csv.
3. **Summarize**: Summarize a data driven insights based on your analysis at the end.



In [27]:
from google.genai import types

base_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[{
        "role": "user",
        "parts": [
            {"text": prompt},
            {
                "inline_data": {
                    "mime_type": "text/csv",
                    "data": data1.to_csv(index=False).encode("utf-8")
                }
            },
            {
                "inline_data": {
                    "mime_type": "text/csv",
                    "data": data3.to_csv(index=False).encode("utf-8")
                }
            },
                        {
                "inline_data": {
                    "mime_type": "text/csv",
                    "data": data2.to_csv(index=False).encode("utf-8")
                }
            }
        ]
    }],
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    ),
)

In [50]:
base_response.candidates[0].content.parts[0]

Part(
  executable_code=ExecutableCode(
    code="""import pandas as pd

# Load the Odisha data
df_odisha = pd.read_csv("input_file_0.csv")

# Load the Assam data
df_assam = pd.read_csv("input_file_1.csv")

print("Odisha Data:")
print(df_odisha.head())
print("\nAssam Data:")
print(df_assam.head())
""",
    language=<Language.PYTHON: 'PYTHON'>
  )
)

In [28]:
for part in base_response.candidates[0].content.parts:
    print(part)
    print("=" * 20)
    print("\n\n")

function_call=None code_execution_result=None executable_code=ExecutableCode(
  code="""import pandas as pd

# Load the Odisha data
df_odisha = pd.read_csv("input_file_0.csv")

# Load the Assam data
df_assam = pd.read_csv("input_file_1.csv")

print("Odisha Data:")
print(df_odisha.head())
print("\nAssam Data:")
print(df_assam.head())
""",
  language=<Language.PYTHON: 'PYTHON'>
) file_data=None function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None



function_call=None code_execution_result=CodeExecutionResult(
  outcome=<Outcome.OUTCOME_OK: 'OUTCOME_OK'>,
  output="""Odisha Data:
  tree_cover_loss_driver      area_ha  carbon_emissions_MgCO2e
0       Hard commodities    25966.917             1.761503e+07
1  Permanent agriculture  1820299.372             1.163551e+09
2                Unknown     3810.596             1.766994e+06
3               Wildfire  2187455.423             5.980315e+08
4                Logging   217133.258          

In [39]:
csv_data = base_response.candidates[0].content.parts[7]

In [47]:
import pandas as pd
from io import BytesIO

df = pd.read_csv(BytesIO(csv_data.inline_data.data))
print(df)

         tree_cover_loss_driver  Odisha_area_ha  Assam_area_ha
0              Hard commodities       25966.917          6.800
1                       Logging      217133.258          2.250
2    Other natural disturbances       53478.238        478.238
3         Permanent agriculture     1820299.372         99.372
4  Settlements & Infrastructure        5799.188          9.180
5          Shifting cultivation       81114.749         14.700
6                       Unknown        3810.596         10.596
7                      Wildfire     2187455.423         55.230


In [31]:
for part in base_response.candidates[0].content.parts:
    print(part)
    print("=" * 20)
    print("\n\n")

function_call=None code_execution_result=None executable_code=ExecutableCode(
  code="""import pandas as pd

# Load the Odisha dataset
df_odisha = pd.read_csv('input_file_0.csv')
print("Odisha Data (input_file_0.csv):")
print(df_odisha.info())
print(df_odisha.head())

# Load the Assam dataset
df_assam = pd.read_csv('input_file_1.csv')
print("\nAssam Data (input_file_1.csv):")
print(df_assam.info())
print(df_assam.head())
""",
  language=<Language.PYTHON: 'PYTHON'>
) file_data=None function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None



function_call=None code_execution_result=CodeExecutionResult(
  outcome=<Outcome.OUTCOME_OK: 'OUTCOME_OK'>,
  output="""Odisha Data (input_file_0.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tree_cover_loss_driver   8 non-null     

In [32]:
for part in base_response.candidates[0].content.parts:
    if part.text is not None:
        print("\nTEXT")
        print(part.text)
        print("=" * 50)
    if part.executable_code is not None:
        print("\nCODE")
        print(part.executable_code.code)
        print("=" * 50)
    if part.code_execution_result is not None:
        print("\nRESULT")
        print(part.code_execution_result.output)
        print("=" * 50)


CODE
import pandas as pd

# Load the Odisha dataset
df_odisha = pd.read_csv('input_file_0.csv')
print("Odisha Data (input_file_0.csv):")
print(df_odisha.info())
print(df_odisha.head())

# Load the Assam dataset
df_assam = pd.read_csv('input_file_1.csv')
print("\nAssam Data (input_file_1.csv):")
print(df_assam.info())
print(df_assam.head())


RESULT
Odisha Data (input_file_0.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tree_cover_loss_driver   8 non-null      object 
 1   area_ha                  8 non-null      float64
 2   carbon_emissions_MgCO2e  8 non-null      float64
dtypes: float64(2), object(1)
memory usage: 324.0+ bytes
None
  tree_cover_loss_driver      area_ha  carbon_emissions_MgCO2e
0       Hard commodities    25966.917             1.761503e+07
1  Permanent agriculture  1820299.372             1.163551

In [14]:
print(base_response.candidates[0].content.parts[-1].text)

Here's a comparison of deforestation by driver between Odisha and Assam for the period 2020-2024:

**Summary of Tree Cover Loss by Driver:**

| State   | tree_cover_loss_driver   | area_ha       | carbon_emissions_MgCO2e |
|:--------|:-------------------------|:--------------|:------------------------|
| Assam   | Hard commodities         | 6.80          | 27.20                   |
| Assam   | Logging                  | 2.25          | 69.22                   |
| Assam   | Other natural disturbances | 478.24        | 21.17                   |
| Assam   | Permanent agriculture    | 99.37         | 72.74                   |
| Assam   | Settlements & Infrastructure | 9.18          | 979.21                  |
| Assam   | Shifting cultivation     | 14.70         | 802.21                  |
| Assam   | Unknown                  | 10.60         | 93.94                   |
| Assam   | Wildfire                 | 55.23         | 90.82                   |
| Odisha  | Hard commodities         | 259

### Generate title & insight

In [60]:
from pydantic import BaseModel, Field

class Insight(BaseModel):
    title: str = Field(description="Clear, descriptive title.")
    insight: str = Field(
        description="Key insight or finding from the analysis that best answers the user query."
    )

prompt = f"""Write a crisp title & summary insights that best answers the user query - keep them data driven.

### User query
Compare deforestation between Assam & Odisha

### Analysis by agent
{base_response.candidates[0].content.parts[-1].text}
"""

print(prompt)

Write a crisp title & summary insights that best answers the user query - keep them data driven.

### User query
Compare deforestation between Assam & Odisha

### Analysis by agent
Deforestation by driver between Odisha and Assam has been compared using the provided tree cover loss data from 2020-2024.

Here is the breakdown of deforestation drivers for both states:

| tree_cover_loss_driver   | Odisha Area (ha)      | Odisha Carbon Emissions (MgCO2e) | Assam Area (ha)     | Assam Carbon Emissions (MgCO2e) |
|:-------------------------|:----------------------|:---------------------------------|:--------------------|:--------------------------------|
| Hard commodities         | 25,966.92             | 17,615,027.09                    | 6.80                | 27.20                           |
| Logging                  | 217,133.26            | 254,215,669.24                   | 2.25                | 69.22                           |
| Other natural disturbances | 53,478.24             |

In [61]:
insight_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config={
        "response_mime_type": "application/json",
        "response_schema": Insight,
    },
)

In [62]:
insight_response.parsed

Insight(title='Deforestation Comparison: Odisha vs. Assam (2020-2024)', insight="Between 2020-2024, Odisha experienced significantly higher deforestation than Assam, with a total tree cover loss of approximately 4.4 million hectares and 2.1 billion MgCO2e in emissions, compared to Assam's 676.37 hectares and 2,156.51 MgCO2e. In Odisha, wildfires and permanent agriculture were the dominant drivers, accounting for 2.19 million ha and 1.82 million ha respectively. In contrast, Assam's relatively minimal tree cover loss was primarily attributed to other natural disturbances (478 ha).")

In [63]:
class ChartData(BaseModel):
    """
    Recharts-compatible data.
    """
    chart_type: str = Field(
        description="Chart type: 'line', 'bar', 'stacked-bar', 'grouped-bar', 'pie', 'area', 'scatter', or 'table'"
    )
    data: List = Field(
        description="Recharts-compatible data array with objects containing key-value pairs"
    )
    x_axis: str = Field(
        description="Name of the field to use for X-axis (for applicable chart types)"
    )
    y_axis: str = Field(
        description="Name of the field to use for Y-axis (for applicable chart types)"
    )
    color_field: str = Field(
        default="",
        description="Optional field name for color grouping/categorization",
    )
    stack_field: str = Field(
        default="",
        description="Field name for stacking data (for stacked-bar charts)",
    )
    group_field: str = Field(
        default="",
        description="Field name for grouping bars (for grouped-bar charts)",
    )
    series_fields: List[str] = Field(
        default=[],
        description="List of field names for multiple data series (for multi-bar charts)",
    )

In [64]:
prompt = f"""Write code to return a data array compatible with recharts format for a suitable chart type that best represents the analysis by agent based on user query.

### User query
Compare deforestation between Assam & Odisha

### Analysis by agent
{base_response.candidates[0].content.parts[-1].text}
"""

In [66]:
print(prompt)

Write code to return a data array compatible with recharts format for a suitable chart type that best represents the analysis by agent based on user query.

### User query
Compare deforestation between Assam & Odisha

### Analysis by agent
Deforestation by driver between Odisha and Assam has been compared using the provided tree cover loss data from 2020-2024.

Here is the breakdown of deforestation drivers for both states:

| tree_cover_loss_driver   | Odisha Area (ha)      | Odisha Carbon Emissions (MgCO2e) | Assam Area (ha)     | Assam Carbon Emissions (MgCO2e) |
|:-------------------------|:----------------------|:---------------------------------|:--------------------|:--------------------------------|
| Hard commodities         | 25,966.92             | 17,615,027.09                    | 6.80                | 27.20                           |
| Logging                  | 217,133.26            | 254,215,669.24                   | 2.25                | 69.22                        

In [71]:
chart_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config={
        "response_mime_type": "application/json",
        "response_schema": ChartData,
    },
)

In [72]:
chart_response.parsed

ChartData(chart_type='grouped-bar', data=[{'tree_cover_loss_driver': 'Hard commodities', 'Odisha Area (ha)': 25966.92, 'Assam Area (ha)': 6.8}, {'tree_cover_loss_driver': 'Logging', 'Odisha Area (ha)': 217133.26, 'Assam Area (ha)': 2.25}, {'tree_cover_loss_driver': 'Other natural disturbances', 'Odisha Area (ha)': 53478.24, 'Assam Area (ha)': 478.24}, {'tree_cover_loss_driver': 'Permanent agriculture', 'Odisha Area (ha)': 1820299.37, 'Assam Area (ha)': 99.37}, {'tree_cover_loss_driver': 'Settlements & Infrastructure', 'Odisha Area (ha)': 5799.19, 'Assam Area (ha)': 9.18}, {'tree_cover_loss_driver': 'Shifting cultivation', 'Odisha Area (ha)': 81114.75, 'Assam Area (ha)': 14.7}, {'tree_cover_loss_driver': 'Unknown', 'Odisha Area (ha)': 3810.6, 'Assam Area (ha)': 10.6}, {'tree_cover_loss_driver': 'Wildfire', 'Odisha Area (ha)': 2187455.42, 'Assam Area (ha)': 55.23}], x_axis='tree_cover_loss_driver', y_axis='Area (ha)', color_field='', stack_field='', group_field='', series_fields=['Odisha

In [70]:
#Todo: Improve the pydantic types, use enum where required - think of ways to reduce this to 2 LLM calls instead of 3.

### CodeAct
Use create_react_agent with just a single tool call

In [18]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool
from langgraph.checkpoint.memory import InMemorySaver

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=5,
)
checkpointer = InMemorySaver()

In [19]:
code_execution_tool = GenAITool(code_execution={})
search = GenAITool(google_search={})
chain = llm.bind_tools(tools=[search, code_execution_tool])

In [20]:
r = chain.invoke("write a program to compute all prime numbers before the date of birth of mahatma gandhi")

In [21]:
print(r.content[1])

```python
def sieve_of_eratosthenes(limit):
    """
    Computes all prime numbers up to a given limit using the Sieve of Eratosthenes.

    Args:
        limit: An integer representing the upper bound (exclusive) for prime numbers.

    Returns:
        A list of prime numbers less than the limit.
    """
    if limit <= 2:
        return []

    # Create a boolean array "is_prime[0..limit]" and initialize
    # all entries it as true. A value in is_prime[i] will
    # finally be false if i is Not a prime, else true.
    is_prime = [True] * limit
    is_prime[0] = is_prime[1] = False  # 0 and 1 are not prime numbers

    # Start checking from p = 2
    p = 2
    while (p * p < limit):
        # If is_prime[p] is still true, then it is a prime
        if is_prime[p]:
            # Update all multiples of p
            for multiple in range(p * p, limit, p):
                is_prime[multiple] = False
        p += 1

    # Collect all prime numbers
    primes = [i for i, prime_status in 

In [22]:
# ToolNode throws error: ValueError: The first argument must be a string or a callable with a __name__ for tool decorator. Got <class 'google.ai.generativelanguage_v1beta.types.content.Tool'>
# agent = create_react_agent(
#     model=llm,
#     tools=[search],
#     prompt="You are an expert data analyst, try to solve problems using the code execution tool using CodeAct pattern.",
#     checkpointer=checkpointer
# )

### ReAct

Simple implementattion - Pass data in prompt, generate analysis by writing code & executing in a sandbox environment

In [18]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_sandbox import PyodideSandbox, PyodideSandboxTool
from langchain_google_genai import ChatGoogleGenerativeAI

In [19]:
box = PyodideSandbox(
    allow_net=True
)

code = """\
import math

a = math.sqrt(100)
print(a)
"""

print(await box.execute(code))

CodeExecutionResult(result=None, stdout='10.0', stderr=None, status='success', execution_time=0.9530489444732666, session_metadata={'created': '2025-10-21T10:57:36.057Z', 'lastModified': '2025-10-21T10:57:36.079Z', 'packages': []}, session_bytes=None)


In [52]:
code = '''
import pandas as pd
import io

odisha_data = """tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
Hard commodities,25966.917,17615027.092
Permanent agriculture,1820299.372,1163551272.738
Unknown,3810.596,1766993.938
Wildfire,2187455.423,598031490.819
Logging,217133.258,254215669.242
Other natural disturbances,53478.238,22167621.917
Settlements & Infrastructure,5799.188,3291979.121
Shifting cultivation,81114.749,51868802.621
"""

assam_data = """tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
Hard commodities,6.8,27.2
Permanent agriculture,99.372,72.738
Unknown,10.596,93.938
Wildfire,55.23,90.819
Logging,2.25,69.22
Other natural disturbances,478.238,21.17
Settlements & Infrastructure,9.18,979.21
Shifting cultivation,14.7,802.21
"""

odisha_df = pd.read_csv(io.StringIO(odisha_data))
assam_df = pd.read_csv(io.StringIO(assam_data))

print()
print()
print("Odisha Deforestation Drivers:")
print(odisha_df.sort_values(by='area_ha', ascending=False).to_csv(index=False))
print("Assam Deforestation Drivers:")
print(assam_df.sort_values(by='area_ha', ascending=False).to_csv(index=False))

# Odisha Analysis
odisha_total_area = odisha_df['area_ha'].sum()
odisha_df['percentage_area'] = (odisha_df['area_ha'] / odisha_total_area) * 100
print("Odisha Deforestation Drivers by Percentage of Area:")
print(odisha_df.sort_values(by='percentage_area', ascending=False).to_csv(index=False))


# Assam Analysis
assam_total_area = assam_df['area_ha'].sum()
assam_df['percentage_area'] = (assam_df['area_ha'] / assam_total_area) * 100
print("Assam Deforestation Drivers by Percentage of Area:")
print(assam_df.sort_values(by='percentage_area', ascending=False).to_csv(index=False))
'''

result = await box.execute(code)

In [53]:
print(result.stdout)

Loading numpy, pandas, python-dateutil, pytz, sixLoaded numpy, pandas, python-dateutil, pytz, sixOdisha Deforestation Drivers:tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2eWildfire,2187455.423,598031490.819Permanent agriculture,1820299.372,1163551272.738Logging,217133.258,254215669.242Shifting cultivation,81114.749,51868802.621Other natural disturbances,53478.238,22167621.917Hard commodities,25966.917,17615027.092Settlements & Infrastructure,5799.188,3291979.121Unknown,3810.596,1766993.938Assam Deforestation Drivers:tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2eOther natural disturbances,478.238,21.17Permanent agriculture,99.372,72.738Wildfire,55.23,90.819Shifting cultivation,14.7,802.21Unknown,10.596,93.938Settlements & Infrastructure,9.18,979.21Hard commodities,6.8,27.2Logging,2.25,69.22Odisha Deforestation Drivers by Percentage of Area:tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e,percentage_areaWildfire,2187455.423,598031490.819,49.77080056523426Permanent a

In [38]:
print(result.stdout)

None


In [60]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=5,
)
code_executor_tool = PyodideSandboxTool(allow_net=True)

agent = create_react_agent(
    model=llm,
    tools=[code_executor_tool],
    prompt="You are an GeoAI data analyst. Help answer user query by writing code & executing using the `python_code_sandbox` tool. Goal is to analyse the dataset, generate insight from data - DONOT plot anything or generate visuals, use print statements to look at insights."
)

I0000 00:00:1760593879.736658   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [25]:
prompt = f"""
User query:
Compare deforestation by driver between Odisha & Assam.

Odisha data:
```csv
{data1.to_csv(index=False)}
```

Assam data:
```csv
{data2.to_csv(index=False)}
```
"""
print(prompt)


User query:
Compare deforestation by driver between Odisha & Assam.

Odisha data:
```csv
tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
Hard commodities,25966.917,17615027.092
Permanent agriculture,1820299.372,1163551272.738
Unknown,3810.596,1766993.938
Wildfire,2187455.423,598031490.819
Logging,217133.258,254215669.242
Other natural disturbances,53478.238,22167621.917
Settlements & Infrastructure,5799.188,3291979.121
Shifting cultivation,81114.749,51868802.621

```

Assam data:
```csv
tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
Hard commodities,6.8,27.2
Permanent agriculture,99.372,72.738
Unknown,10.596,93.938
Wildfire,55.23,90.819
Logging,2.25,69.22
Other natural disturbances,478.238,21.17
Settlements & Infrastructure,9.18,979.21
Shifting cultivation,14.7,802.21

```



In [62]:
result = await agent.ainvoke({
    "messages": [{"role": "user", "content": prompt}]
})

I0000 00:00:1760593890.707132   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1760593905.452905   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1760593915.154189   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1760593928.182637   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1760593944.877677   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1760593961.399409   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1760593976.797858   41310 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [64]:
import pandas as pd
import io

odisha_data = """tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
Hard commodities,25966.917,17615027.092
Permanent agriculture,1820299.372,1163551272.738
Unknown,3810.596,1766993.938
Wildfire,2187455.423,598031490.819
Logging,217133.258,254215669.242
Other natural disturbances,53478.238,22167621.917
Settlements & Infrastructure,5799.188,3291979.121
Shifting cultivation,81114.749,51868802.621
"""

assam_data = """tree_cover_loss_driver,area_ha,carbon_emissions_MgCO2e
Hard commodities,6.8,27.2
Permanent agriculture,99.372,72.738
Unknown,10.596,93.938
Wildfire,55.23,90.819
Logging,2.25,69.22
Other natural disturbances,478.238,21.17
Settlements & Infrastructure,9.18,979.21
Shifting cultivation,14.7,802.21
"""

odisha_df = pd.read_csv(io.StringIO(odisha_data))
assam_df = pd.read_csv(io.StringIO(assam_data))

print("Odisha Deforestation Drivers:")
print(odisha_df.sort_values(by='area_ha', ascending=False))
print("\nAssam Deforestation Drivers:")
print(assam_df.sort_values(by='area_ha', ascending=False))

# Odisha Analysis
odisha_total_area = odisha_df['area_ha'].sum()
odisha_df['percentage_area'] = (odisha_df['area_ha'] / odisha_total_area) * 100
print("\nOdisha Deforestation Drivers by Percentage of Area:")
print(odisha_df.sort_values(by='percentage_area', ascending=False))


# Assam Analysis
assam_total_area = assam_df['area_ha'].sum()
assam_df['percentage_area'] = (assam_df['area_ha'] / assam_total_area) * 100
print("\nAssam Deforestation Drivers by Percentage of Area:")
print(assam_df.sort_values(by='percentage_area', ascending=False))

Odisha Deforestation Drivers:
         tree_cover_loss_driver      area_ha  carbon_emissions_MgCO2e
3                      Wildfire  2187455.423             5.980315e+08
1         Permanent agriculture  1820299.372             1.163551e+09
4                       Logging   217133.258             2.542157e+08
7          Shifting cultivation    81114.749             5.186880e+07
5    Other natural disturbances    53478.238             2.216762e+07
0              Hard commodities    25966.917             1.761503e+07
6  Settlements & Infrastructure     5799.188             3.291979e+06
2                       Unknown     3810.596             1.766994e+06

Assam Deforestation Drivers:
         tree_cover_loss_driver  area_ha  carbon_emissions_MgCO2e
5    Other natural disturbances  478.238                   21.170
1         Permanent agriculture   99.372                   72.738
3                      Wildfire   55.230                   90.819
7          Shifting cultivation   14.700      

In [23]:
# for msg in result["messages"]:
#     msg.pretty_print()

### E2B Sandbox

In [34]:
from e2b_code_interpreter import Sandbox

In [35]:
sbx = Sandbox.create() # By default the sandbox is alive for 5 minutes
execution = sbx.run_code("print('hello world')") # Execute Python inside the sandbox
print(execution.logs)

Logs(stdout: ['hello world\n'], stderr: [])


In [36]:
execution

Execution(Results: [], Logs: Logs(stdout: ['hello world\n'], stderr: []), Error: None)

### ReAct + llm-sandbox

In [8]:
%%time

from llm_sandbox import SandboxSession

with SandboxSession(
    lang="python",
    verbose=True,
    keep_template=True,
    workdir="/sandbox",
    skip_environment_setup=False,
    image="quay.io/jupyter/scipy-notebook" # "ghcr.io/vndee/sandbox-python-311-bullseye"
) as session:
    result = session.run("""
import numpy as np
import pandas as pd

# Create an array
arr = np.array([1, 2, 3, 4, 5])
df = pd.DataFrame(arr)
print(f"Array: {arr}")
print(f"Mean: {np.mean(arr)}")
print(f"Dataframe: {df}")
    """)

    print(result.stdout)

2025-10-28 10:18:30 - llm_sandbox.core.session_base - INFO - Using local Docker context since client is not provided.
2025-10-28 10:18:30 - llm_sandbox.core.session_base - INFO - Using local image quay.io/jupyter/scipy-notebook:latest
2025-10-28 10:18:30 - llm_sandbox.core.session_base - INFO - Executing command: mkdir -p /sandbox
2025-10-28 10:18:30 - llm_sandbox.core.session_base - INFO - Executing command: python -m venv --system-site-packages /tmp/venv
2025-10-28 10:18:31 - llm_sandbox.core.session_base - INFO - Executing command: mkdir -p /tmp/pip_cache
2025-10-28 10:18:31 - llm_sandbox.core.session_base - INFO - Executing command: /tmp/venv/bin/pip install --upgrade pip --cache-dir /tmp/pip_cache
2025-10-28 10:18:32 - llm_sandbox.core.session_base - INFO - STDOUT: Requirement already satisfied: pip in /tmp/venv/lib/python3.13/site-packages (25.3)

2025-10-28 10:18:32 - llm_sandbox.core.session_base - INFO - Copying /var/folders/pn/__395l652sg6ck9cy_gds59m0000gn/T/tmp_c1oqm5q.py t

Array: [1 2 3 4 5]
Mean: 3.0
Dataframe:    0
0  1
1  2
2  3
3  4
4  5



2025-10-28 10:18:32 - llm_sandbox.core.session_base - INFO - Stopped and removed container


CPU times: user 62.3 ms, sys: 20.6 ms, total: 82.9 ms
Wall time: 2.12 s


In [9]:
from langchain.tools import BaseTool
from llm_sandbox import SandboxSession

from langchain_core.tools import tool

@tool("python_sandbox")
def python_sandbox(code: str) -> str:
    """Execute Python code in a secure sandbox.

    code: python code to be executed inside a sandbox container
    """
    with SandboxSession(
        lang="python",
        verbose=True,
        keep_template=True,
        workdir="/sandbox",
        skip_environment_setup=False,
        image="quay.io/jupyter/scipy-notebook" # "ghcr.io/vndee/sandbox-python-311-bullseye"
    ) as session:
        result = session.run(code)
        return result.stdout if result.exit_code == 0 else result.stderr

In [10]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=5,
)

agent = create_react_agent(
    model=llm,
    tools=[python_sandbox],
    prompt="You are an GeoAI data analyst. Help answer user query by writing code & executing using the `python_code_sandbox` tool. Goal is to analyse the dataset, generate insight from data - DONOT plot anything or generate visuals, use print statements to look at insights."
)

In [11]:
# PASS DATA inside PROMPT

# prompt = f"""
# User query:
# Compare deforestation by driver between Odisha & Assam.

# Odisha data:
# ```csv
# {data1.to_csv(index=False)}
# ```

# Assam data:
# ```csv
# {data2.to_csv(index=False)}
# ```
# """
# print(prompt)

# result = agent.invoke({
#     "messages": [{"role": "user", "content": prompt}]
# })

# for msg in result["messages"]:
#     msg.pretty_print()

#### Access to file system

In [12]:
from typing import List
from IPython.core.debugger import set_trace

@tool("python_sandbox")
def python_sandbox(code: str, files: List[str]) -> str:
    """Execute Python code in a secure sandbox.

    code: python code to be executed inside a sandbox container
    files: list of file names accessible inside the sandbox container
    """
    print("CODE")
    print(code)
    print(f"FILES: {files}")
    with SandboxSession(
        lang="python",
        verbose=True,
        keep_template=True,
        commit_container=True,
        workdir="/sandbox",
        skip_environment_setup=False,
        image="quay.io/jupyter/scipy-notebook" # "ghcr.io/vndee/sandbox-python-311-bullseye"
    ) as session:
        # copy files from local to container
        for file in files:
            print(f"copying file {file} -> /sandbox/{file}.")
            session.copy_to_runtime(file, f"/sandbox/{file}")
        print("FILE COPY DONE")
        
        # Run code
        result = session.run(code)
        print("CODE RUN DONE")

        # Check if chart_data.csv was created using Python code
        check_code = """
from pathlib import Path
print(Path('/sandbox/chart_data.csv').exists())
"""
        check_result = session.run(check_code)
        
        if check_result.stdout.strip() == 'True':
            print(f"saving file from /sandbox/chart_data.csv -> chart_data.csv")
            session.copy_from_runtime("/sandbox/chart_data.csv", "chart_data.csv")
        print("CHECK FOR CHART_DATA DONE")
        
        return result.stdout if result.exit_code == 0 else result.stderr

In [13]:
prompt = f"""
User query:
Compare deforestation by driver between Odisha & Assam.

Note:
You have access to the following files inside the container.
- odisha.csv
- assam.csv

Use these files to read the datasets, they are present inside `/sandbox` directory.
```
"""
print(prompt)


User query:
Compare deforestation by driver between Odisha & Assam.

Note:
You have access to the following files inside the container.
- odisha.csv
- assam.csv

Use these files to read the datasets, they are present inside `/sandbox` directory.
```



In [ ]:
from langchain_anthropic import ChatAnthropic

sonnet = ChatAnthropic(model_name="claude-sonnet-4-20250514")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=5,
)

In [35]:
test_prompt = """You are an expert Analyst. 
Help answer user query by writing code & executing using the python_sandbox tool. 
Goal is to analyse the dataset, generate insight from data - DONOT plot anything or generate visuals, use print statements to look at insights. 
Notes: 
- Before starting your analysis, look at head() & info() of the datasets, to know the column names & understand the data formats & units.
- Recommend the a visualization to best represent the insights in recharts.js format, fill in below:
    - chart_type: pick one from - 'line', 'bar', 'stacked-bar', 'grouped-bar', 'pie', 'area', 'scatter', or 'table'
    - x_axis: name of field for x_axis
    - y_axis: name of field for y_axis
    - color_field: Optional - field name for color grouping/categorization
    - stack_field: Optional - field name for stacking data (for stacked-bar charts)
    - group_field: Optional - field name for grouping bars (for grouped-bar charts)
- Write code to save the data for the recommended chart type in recharts.js data format as '/sandbox/chart_data.csv'."""


working_prompt = """You are an expert Analyst. 
Help answer user query by writing code & executing using the python_sandbox tool. 
Goal is to analyse the dataset, generate insight from data - DONOT plot anything or generate visuals, use print statements to look at insights. 
Notes: 
- Before starting your analysis, look at head() & info() of the datasets, to know the column names & understand the data formats & units.
- Recommend a chart type & save the chart data as `/sandbox/chart_data.csv`
"""

new_prompt = """You are an expert Analyst helping users analyze datasets via code execution using the 'python_sandbox' tool.

Workflow:
1. **Explore datasets first**: Identify datasets relevant to user query, load them, use head(), info(), describe() to understand structure, columns, dtypes, and units. Never assume column names or formats.
2. **Analyze**: Write code to extract insights using pandas operations and print statements. DO NOT create plots or visualizations.
3. **Output**: Recommend an appropriate chart type and save the prepared chart data to `/sandbox/chart_data.csv`."""

agent = create_react_agent(
    model=llm,
    tools=[python_sandbox],
    prompt=new_prompt
)

In [36]:
result = agent.invoke({
    "messages": [{"role": "user", "content": prompt}]
})

2025-10-28 10:46:19 - llm_sandbox.core.session_base - INFO - Using local Docker context since client is not provided.
2025-10-28 10:46:19 - llm_sandbox.core.session_base - INFO - Using local image quay.io/jupyter/scipy-notebook:latest
2025-10-28 10:46:20 - llm_sandbox.core.session_base - INFO - Executing command: mkdir -p /sandbox
2025-10-28 10:46:20 - llm_sandbox.core.session_base - INFO - Executing command: python -m venv --system-site-packages /tmp/venv


CODE
import pandas as pd

# Load the datasets
odisha_df = pd.read_csv('/sandbox/odisha.csv')
assam_df = pd.read_csv('/sandbox/assam.csv')

# Display info and head for Odisha
print("Odisha Dataset:")
print(odisha_df.info())
print(odisha_df.head())

# Display info and head for Assam
print("\nAssam Dataset:")
print(assam_df.info())
print(assam_df.head())
FILES: ['odisha.csv', 'assam.csv']


2025-10-28 10:46:20 - llm_sandbox.core.session_base - INFO - Executing command: mkdir -p /tmp/pip_cache
2025-10-28 10:46:20 - llm_sandbox.core.session_base - INFO - Executing command: /tmp/venv/bin/pip install --upgrade pip --cache-dir /tmp/pip_cache
2025-10-28 10:46:21 - llm_sandbox.core.session_base - INFO - STDOUT: Requirement already satisfied: pip in /tmp/venv/lib/python3.13/site-packages (25.3)

2025-10-28 10:46:21 - llm_sandbox.core.session_base - INFO - Copying odisha.csv to /sandbox/odisha.csv
2025-10-28 10:46:21 - llm_sandbox.core.session_base - INFO - Copying assam.csv to /sandbox/assam.csv
2025-10-28 10:46:21 - llm_sandbox.core.session_base - INFO - Copying /var/folders/pn/__395l652sg6ck9cy_gds59m0000gn/T/tmpyvhjaa8y.py to /sandbox/df0dd80878604b40a099f36664dd671f.py
2025-10-28 10:46:21 - llm_sandbox.core.session_base - INFO - Executing command: /tmp/venv/bin/python /sandbox/df0dd80878604b40a099f36664dd671f.py


copying file odisha.csv -> /sandbox/odisha.csv.
copying file assam.csv -> /sandbox/assam.csv.
FILE COPY DONE


2025-10-28 10:46:21 - llm_sandbox.core.session_base - INFO - STDOUT: Odisha Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tree_cover_loss_driver   8 non-null      object 
 1   area_ha                  8 non-null      float64
 2   carbon_emissions_MgCO2e  8 non-null      float64
dtypes: float64(2), object(1)
memory usage: 324.0+ bytes
None
  tree_cover_loss_driver      area_ha  carbon_emissions_MgCO2e
0       Hard commodities    25966.917             1.761503e+07
1  Permanent agriculture  1820299.372             1.163551e+09
2                Unknown     3810.596             1.766994e+06
3               Wildfire  2187455.423             5.980315e+08
4                Logging   217133.258             2.542157e+08

Assam Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3

...

2025-10-28 10:46:29 - llm_sandbox.core.session_base - INFO - Executing command: mkdir -p /tmp/pip_cache
2025-10-28 10:46:29 - llm_sandbox.core.session_base - INFO - Executing command: /tmp/venv/bin/pip install --upgrade pip --cache-dir /tmp/pip_cache
2025-10-28 10:46:29 - llm_sandbox.core.session_base - INFO - STDOUT: Requirement already satisfied: pip in /tmp/venv/lib/python3.13/site-packages (25.3)

2025-10-28 10:46:29 - llm_sandbox.core.session_base - INFO - Copying odisha.csv to /sandbox/odisha.csv
2025-10-28 10:46:29 - llm_sandbox.core.session_base - INFO - Copying assam.csv to /sandbox/assam.csv
2025-10-28 10:46:29 - llm_sandbox.core.session_base - INFO - Copying /var/folders/pn/__395l652sg6ck9cy_gds59m0000gn/T/tmpsaexhca_.py to /sandbox/cd8856f3f4eb477cbebbab882ff66be0.py
2025-10-28 10:46:29 - llm_sandbox.core.session_base - INFO - Executing command: /tmp/venv/bin/python /sandbox/cd8856f3f4eb477cbebbab882ff66be0.py


copying file odisha.csv -> /sandbox/odisha.csv.
copying file assam.csv -> /sandbox/assam.csv.
FILE COPY DONE


2025-10-28 10:46:30 - llm_sandbox.core.session_base - INFO - STDOUT: Comparison of Deforestation by Driver between Odisha and Assam (in hectares):
         tree_cover_loss_driver  Odisha_area_ha  Assam_area_ha
0              Hard commodities       25966.917          6.800
1         Permanent agriculture     1820299.372         99.372
2                       Unknown        3810.596         10.596
3                      Wildfire     2187455.423         55.230
4                       Logging      217133.258          2.250
5    Other natural disturbances       53478.238        478.238
6  Settlements & Infrastructure        5799.188          9.180
7          Shifting cultivation       81114.749         14.700

2025-10-28 10:46:30 - llm_sandbox.core.session_base - INFO - Copying /var/folders/pn/__395l652sg6ck9cy_gds59m0000gn/T/tmpm45mi4ww.py to /sandbox/d02ede93b6ee48208d204f92b952286b.py
2025-10-28 10:46:30 - llm_sandbox.core.session_base - INFO - Executing command: /tmp/venv/bin/python /sa

CODE RUN DONE
saving file from /sandbox/chart_data.csv -> chart_data.csv
CHECK FOR CHART_DATA DONE


2025-10-28 10:46:30 - llm_sandbox.core.session_base - INFO - Committed container as image quay.io/jupyter/scipy-notebook:latest
2025-10-28 10:46:30 - llm_sandbox.core.session_base - INFO - Stopped and removed container


In [37]:
result

{'messages': [HumanMessage(content='\nUser query:\nCompare deforestation by driver between Odisha & Assam.\n\nNote:\nYou have access to the following files inside the container.\n- odisha.csv\n- assam.csv\n\nUse these files to read the datasets, they are present inside `/sandbox` directory.\n```\n', additional_kwargs={}, response_metadata={}, id='96a9ff6c-847b-43e5-9250-af196b3e1d42'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'python_sandbox', 'arguments': '{"files": ["odisha.csv", "assam.csv"], "code": "import pandas as pd\\n\\n# Load the datasets\\nodisha_df = pd.read_csv(\'/sandbox/odisha.csv\')\\nassam_df = pd.read_csv(\'/sandbox/assam.csv\')\\n\\n# Display info and head for Odisha\\nprint(\\"Odisha Dataset:\\")\\nprint(odisha_df.info())\\nprint(odisha_df.head())\\n\\n# Display info and head for Assam\\nprint(\\"\\\\nAssam Dataset:\\")\\nprint(assam_df.info())\\nprint(assam_df.head())"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safet

In [38]:
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================


User query:
Compare deforestation by driver between Odisha & Assam.

Note:
You have access to the following files inside the container.
- odisha.csv
- assam.csv

Use these files to read the datasets, they are present inside `/sandbox` directory.
```

================================== Ai Message ==================================
Tool Calls:
  python_sandbox (d33312c4-b731-4050-834a-a7eb251c194f)
 Call ID: d33312c4-b731-4050-834a-a7eb251c194f
  Args:
    files: ['odisha.csv', 'assam.csv']
    code: import pandas as pd

# Load the datasets
odisha_df = pd.read_csv('/sandbox/odisha.csv')
assam_df = pd.read_csv('/sandbox/assam.csv')

# Display info and head for Odisha
print("Odisha Dataset:")
print(odisha_df.info())
print(odisha_df.head())

# Display info and head for Assam
print("\nAssam Dataset:")
print(assam_df.info())
print(assam_df.head())
================================= Tool Message =================

### Generate structured data for ChartInsight

In [132]:
print()

Here are the insights from the analysis of deforestation drivers in Odisha and Assam:

**Comparison of Deforestation by Driver (in hectares):**

| Tree Cover Loss Driver       |   Odisha (ha) |   Assam (ha) |
|:-----------------------------|--------------:|-------------:|
| Hard commodities             |      25966.92 |         6.80 |
| Permanent agriculture        |    1820299.37 |        99.37 |
| Unknown                      |       3810.60 |        10.60 |
| Wildfire                     |    2187455.42 |        55.23 |
| Logging                      |     217133.26 |         2.25 |
| Other natural disturbances   |      53478.24 |       478.24 |
| Settlements & Infrastructure |       5799.19 |         9.18 |
| Shifting cultivation         |      81114.75 |        14.70 |

**Key Insights:**

*   In Odisha, the primary driver of deforestation is **Wildfire**, accounting for **2,187,455.42 hectares**.
*   In Assam, the primary driver of deforestation is **Other natural disturbances**, 

In [40]:
import pandas as pd

chart_data_csv = pd.read_csv("chart_data.csv")
chart_data_csv

,tree_cover_loss_driver,Odisha_area_ha,Assam_area_ha
0,Hard commodities,25966.917,6.800
1,Permanent agriculture,1820299.372,99.372
2,Unknown,3810.596,10.596
3,Wildfire,2187455.423,55.230
4,Logging,217133.258,2.250
5,Other natural disturbances,53478.238,478.238
6,Settlements & Infrastructure,5799.188,9.180
7,Shifting cultivation,81114.749,14.700


In [41]:
prompt = f"""Based on analysis done by an expert & data saved for visualization, generate structured response.

### Analysis
{result["messages"][-1].content}

### Saved chart data - head 5 rows
{chart_data_csv.head().to_csv(index=False)}
"""

print(prompt)

Based on analysis done by an expert & data saved for visualization, generate structured response.

### Analysis
Here is a comparison of deforestation drivers between Odisha and Assam:

| tree_cover_loss_driver | Odisha (hectares) | Assam (hectares) |
| :--- | :--- | :--- |
| Wildfire | 2,187,455.42 | 55.23 |
| Permanent agriculture | 1,820,299.37 | 99.37 |
| Logging | 217,133.26 | 2.25 |
| Shifting cultivation | 81,114.75 | 14.70 |
| Other natural disturbances | 53,478.24 | 478.24 |
| Hard commodities | 25,966.92 | 6.80 |
| Settlements & Infrastructure | 5,799.19 | 9.18 |
| Unknown | 3,810.60 | 10.60 |

**Analysis:**
The primary drivers of deforestation in Odisha are Wildfire and Permanent agriculture, with both contributing to over a million hectares of tree cover loss. In contrast, Assam's deforestation figures are significantly lower across all categories, with Other natural disturbances being the most impactful driver.

For a visual comparison, a **Grouped Bar Chart** is recommende

In [45]:
from pydantic import BaseModel, Extra

class ChartInsight(BaseModel, extra=Extra.allow):
    """
    Represents a chart-based insight with Recharts-compatible data.
    """
    title: str = Field(description="Clear, descriptive title for the chart")
    chart_type: str = Field(
        description="Chart type: 'line', 'bar', 'stacked-bar', 'grouped-bar', 'pie', 'area', 'scatter', or 'table'"
    )
    insight: str = Field(
        description="Key insight or finding that this chart reveals (2-3 sentences)"
    )
    x_axis: str = Field(
        description="Name of the field to use for X-axis (for applicable chart types)"
    )
    y_axis: str = Field(
        description="Name of the field to use for Y-axis (for applicable chart types)"
    )
    color_field: str = Field(
        default="",
        description="Optional field name for color grouping/categorization",
    )
    stack_field: str = Field(
        default="",
        description="Field name for stacking data (for stacked-bar charts)",
    )
    group_field: str = Field(
        default="",
        description="Field name for grouping bars (for grouped-bar charts)",
    )
    series_fields: List[str] = Field(
        default=[],
        description="List of field names for multiple data series (for multi-bar charts)",
    )
    follow_up_suggestions: List[str] = Field(
        description="List of 2-3 follow-up prompt suggestions for additional analysis"
    )

agent2 = llm.with_structured_output(ChartInsight)

chart_insight = agent2.invoke(prompt)

/var/folders/pn/__395l652sg6ck9cy_gds59m0000gn/T/ipykernel_28136/2641312017.py:3: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  class ChartInsight(BaseModel, extra=Extra.allow):
Key 'additionalProperties' is not supported in schema, ignoring


In [46]:
chart_insight.data = chart_data_csv.to_dict(orient="records")

In [47]:
chart_insight

ChartInsight(title='Deforestation Drivers: Odisha vs. Assam', chart_type='grouped-bar', insight="The primary drivers of deforestation in Odisha are Wildfire and Permanent agriculture, with both contributing to over a million hectares of tree cover loss. In contrast, Assam's deforestation figures are significantly lower across all categories, with Other natural disturbances being the most impactful driver.", x_axis='tree_cover_loss_driver', y_axis='Area (hectares)', color_field='', stack_field='', group_field='tree_cover_loss_driver', series_fields=['Odisha_area_ha', 'Assam_area_ha'], follow_up_suggestions=['Show the trend of wildfire-driven deforestation in Odisha over the last decade.', "What are the primary crops contributing to 'Permanent agriculture' in Odisha?", 'Compare the deforestation drivers of Assam with other northeastern states.'], data=[{'tree_cover_loss_driver': 'Hard commodities', 'Odisha_area_ha': 25966.917, 'Assam_area_ha': 6.8}, {'tree_cover_loss_driver': 'Permanent 

In [123]:
chart_insight = result["structured_response"]

In [124]:
chart_insight

ChartInsight(title='Deforestation by Driver: Odisha vs. Assam', chart_type='grouped-bar', insight="Odisha has significantly higher deforestation (4.4M ha) compared to Assam (676 ha). The primary driver in Odisha is Wildfire (49.8%), followed by Permanent Agriculture (41.4%). In Assam, 'Other natural disturbances' is the main cause (70.7%), with a much smaller contribution from agriculture (14.7%).", x_axis='Deforestation Driver', y_axis='Area (hectares)', color_field='', stack_field='', group_field='State', series_fields=['Odisha', 'Assam'], follow_up_suggestions=['What is the trend of deforestation in Odisha over the past decade?', "Which specific 'other natural disturbances' are most prevalent in Assam?", 'Show me a breakdown of carbon emissions by deforestation driver for both states.'])

In [125]:
!cat chart_data.csv

tree_cover_loss_driver,odisha_area_ha,assam_area_ha
Hard commodities,25966.917,6.8
Logging,217133.258,2.25
Other natural disturbances,53478.238,478.238
Permanent agriculture,1820299.372,99.372
Settlements & Infrastructure,5799.188,9.18
Shifting cultivation,81114.749,14.7
Unknown,3810.596,10.596
Wildfire,2187455.423,55.23


In [33]:
print(result["messages"][-1].content)

Here's a summary of the comparison between deforestation drivers in Odisha and Assam:

**Key Insights:**

*   **Overall Deforestation:** Odisha has significantly higher deforestation across all drivers compared to Assam. The total deforested area in Odisha is approximately 4.4 million hectares, while in Assam, it is about 676 hectares.
*   **Primary Drivers in Odisha:** The main drivers of deforestation in Odisha are Wildfire (2.19 million ha) and Permanent agriculture (1.82 million ha), which together account for the vast majority of the state's tree cover loss.
*   **Primary Drivers in Assam:** In Assam, the leading causes of deforestation are Other natural disturbances (478 ha) and Permanent agriculture (99 ha), although the overall scale is much smaller than in Odisha.
*   **Carbon Emissions:** The carbon emissions from deforestation are also substantially higher in Odisha. Permanent agriculture and Wildfire are the largest contributors to carbon emissions in Odisha, while in Assam

In [34]:
result

{'messages': [HumanMessage(content='\nUser query:\nCompare deforestation by driver between Odisha & Assam.\n\nNote:\nYou have access to the following files inside the container.\n- odisha.csv\n- assam.csv\n\nUse these files to read the datasets, they are present inside `/sandbox` directory.\n```\n', additional_kwargs={}, response_metadata={}, id='91934ae2-5b9a-4e21-bf2a-2dd28be0d951'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'python_sandbox', 'arguments': '{"files": ["odisha.csv", "assam.csv"], "code": "import pandas as pd\\n\\n# Load the datasets\\nodisha_df = pd.read_csv(\'/sandbox/odisha.csv\')\\nassam_df = pd.read_csv(\'/sandbox/assam.csv\')\\n\\n# Inspect the datasets\\nprint(\\"Odisha Data Head:\\")\\nprint(odisha_df.head())\\nprint(\\"\\\\nOdisha Data Info:\\")\\nodisha_df.info()\\n\\nprint(\\"\\\\nAssam Data Head:\\")\\nprint(assam_df.head())\\nprint(\\"\\\\nAssam Data Info:\\")\\nassam_df.info()"}'}}, response_metadata={'prompt_feedback': {'block_r

In [58]:
from pydantic import BaseModel, Field

class Insight(BaseModel):
    title: str = Field(description="Clear, descriptive title.")
    insight: str = Field(
        description="Key insight or finding from the analysis that best answers the user query."
    )

insight_prompt = f"""Write a crisp title & summary insights that best answers the user query - keep them data driven.

### User query
Compare deforestation by driver between Assam & Odisha.

### Analysis done by agent
{result['messages'][-1].content}
"""

print(insight_prompt)

Write a crisp title & summary insights that best answers the user query - keep them data driven.

### User query
Compare deforestation by driver between Assam & Odisha.

### Analysis done by agent
Here's a comparison of deforestation by driver between Odisha and Assam:

**Key Findings:**

*   **Overall Deforestation:** Odisha experiences significantly higher deforestation (4,395,057.74 hectares) compared to Assam (676.37 hectares).
*   **Major Drivers in Odisha:** The primary drivers of deforestation in Odisha are Wildfire (2,187,455.42 hectares) and Permanent agriculture (1,820,299.37 hectares).
*   **Major Drivers in Assam:** In Assam, the leading cause of deforestation is "Other natural disturbances" (478.24 hectares), although the overall deforestation is very low compared to Odisha.

**Deforestation by Driver (in hectares):**

| Tree Cover Loss Driver       | Odisha          | Assam    |
| ---------------------------- | --------------- | -------- |
| Wildfire                     |

In [60]:
insight_response = (llm
    .with_structured_output(Insight)
    .invoke(insight_prompt)
)

In [61]:
insight_response

Insight(title="Odisha's Deforestation, Fueled by Wildfire & Agriculture, Vastly Exceeds Assam's", insight="Odisha experiences substantially higher deforestation at 4,395,057 hectares compared to Assam's 676 hectares. The primary drivers in Odisha are Wildfire (2,187,455 ha) and Permanent Agriculture (1,820,299 ha). In contrast, Assam's minimal deforestation is mainly attributed to 'Other natural disturbances' (478 ha).")

In [79]:
class ChartData(BaseModel):
    """
    Recharts-compatible data.
    """
    chart_type: str = Field(
        description="Chart type: 'line', 'bar', 'stacked-bar', 'grouped-bar', 'pie', 'area', 'scatter', or 'table'"
    )
    data: List = Field(
        description="Recharts-compatible data array with objects containing key-value pairs"
    )
    x_axis: str = Field(
        description="Name of the field to use for X-axis (for applicable chart types)"
    )
    y_axis: str = Field(
        description="Name of the field to use for Y-axis (for applicable chart types)"
    )
    color_field: str = Field(
        default="",
        description="Optional field name for color grouping/categorization",
    )
    stack_field: str = Field(
        default="",
        description="Field name for stacking data (for stacked-bar charts)",
    )
    group_field: str = Field(
        default="",
        description="Field name for grouping bars (for grouped-bar charts)",
    )
    series_fields: List[str] = Field(
        default=[],
        description="List of field names for multiple data series (for multi-bar charts)",
    )

In [80]:
chart_prompt = f"""Write code to return a data array compatible with recharts format for a suitable chart type that best represents the analysis by agent based on user query.

### User query
Compare deforestation between Assam & Odisha.

### Analysis by agent
{result['messages'][-1].content}
"""

print(chart_prompt)

Write code to return a data array compatible with recharts format for a suitable chart type that best represents the analysis by agent based on user query.

### User query
Compare deforestation between Assam & Odisha.

### Analysis by agent
Here's a comparison of deforestation by driver between Odisha and Assam:

**Key Findings:**

*   **Overall Deforestation:** Odisha experiences significantly higher deforestation (4,395,057.74 hectares) compared to Assam (676.37 hectares).
*   **Major Drivers in Odisha:** The primary drivers of deforestation in Odisha are Wildfire (2,187,455.42 hectares) and Permanent agriculture (1,820,299.37 hectares).
*   **Major Drivers in Assam:** In Assam, the leading cause of deforestation is "Other natural disturbances" (478.24 hectares), although the overall deforestation is very low compared to Odisha.

**Deforestation by Driver (in hectares):**

| Tree Cover Loss Driver       | Odisha          | Assam    |
| ---------------------------- | --------------- |

In [81]:
chart_response = (llm
    .with_structured_output(ChartData, method="json_mode")
    .invoke(chart_prompt)
)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.generation_config.response_schema.properties[data].items: missing field.


In [77]:
chart_response

ChartData(chart_type='grouped-bar', data=[{}, {}, {}, {}, {}, {}, {}, {}], x_axis='driver', y_axis='Hectares', color_field='', stack_field='', group_field='driver', series_fields=['Odisha', 'Assam'])